# Notebook 1: Initial dataset exploration

In [1]:
#imports
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
%matplotlib inline
sns.set_palette("Set2")

In [3]:
#load data
raw_ds=pd.read_csv(r"..\data\raw\investigator_nacc50.csv")
dd=pd.read_csv(r"..\docs\rdd_datadictionary_uds.csv")

C:\Users\viceva\miniconda3\envs\AD\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (20,22,24,26,28,41,44,46,48,51,61,63,65,67,69,71,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,134,156,165,176,179,217,220,222,224,226,228,230,232,234,236,238,240,242,244,246,248,250,252,254,256,258,260,262,264,266,268,270,272,382,397,399,401,421,423,432,445,454,494,568,576,601,637,653,670,673,678,690,701,707,709,772,773,781,792,794,796,798,804,865,920,921,922,930,931,932,933,943,965,968,971) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Data sample, columns & shape

In [ ]:
raw_ds.head()

In [ ]:
list(raw_ds)

In [ ]:
raw_ds.shape

## Is primary and presumptive diagnosis in dataset of ALzheimer's (1) identical?

In [ ]:

print(sum(raw_ds['NACCALZD']==1)) #Presumptive etiologic diagnosis (could be contributing)
print(sum(raw_ds['NACCETPR']==1)) #Primary etiologic diagnosis

In [ ]:
plt.hist(raw_ds['NACCALZD']);

### Is not identical, we will use the more restrictive NACCETPR

In [ ]:
#NACCUDSD: 3 = MCI; 4 = Dementia; NACCETPR=1 AD; is this last one = to NACCALZD=1? no. see above
MCI_index=np.unique(raw_ds.loc[(raw_ds['NACCUDSD']==3) & ((raw_ds['NACCETPR']==1) | (raw_ds['NACCETPR']==99)),['NACCID']])
Dem_index=np.unique(raw_ds.loc[(raw_ds['NACCUDSD']==4) & (raw_ds['NACCETPR']==1),['NACCID']])
combi_i=set(MCI_index).intersection(Dem_index)
len(combi_i)


In [ ]:
AD_MCI=raw_ds.loc[raw_ds['NACCID'].isin(combi_i)]
#db of etiol AD patients that had at one point MCI and developed dementia


In [ ]:
#add subset of etiol AD patients that had at one point MCI and did not develop dementia
nod_index=set(MCI_index).difference(combi_i)
AD_MCI=pd.concat([AD_MCI,raw_ds.loc[raw_ds['NACCID'].isin(nod_index)]])
len(nod_index)

## Add visit date, create target and interval between visits columns


In [ ]:
#add visit date and create target
AD_MCI['visitdate']=pd.to_datetime(dict(year=AD_MCI.VISITYR, month=AD_MCI.VISITMO, day=AD_MCI.VISITDAY))
scores=np.concatenate((np.ones(len(combi_i)),np.zeros(len(nod_index))))
idi=list(combi_i)+list(nod_index)
op={'NACCID':idi, 'target':scores}
output= pd.DataFrame(op)

In [ ]:
#add interval between visits, with last visit being NAN. This is very inneficient with the foor loop
visit_interval=np.array([])
for i in np.unique(AD_MCI['NACCID']):
    temp=AD_MCI[['NACCVNUM','NACCAVST','visitdate']].loc[AD_MCI['NACCID'] == i]
    for n in temp['NACCVNUM']:
        if n not in temp['NACCAVST'].values:
            visit_next=(temp['visitdate'].loc[temp['NACCVNUM'] == n+1]).values
            visit_now=(temp['visitdate'].loc[temp['NACCVNUM'] == n])
            visit_interval=np.append(visit_interval,(visit_next-visit_now).dt.days)
        else:
            visit_interval=np.append(visit_interval,np.nan)

In [ ]:
AD_MCI['visit_interval']=visit_interval

In [ ]:
#lets map the output to the main
AD_MCI=AD_MCI.join(output.set_index('NACCID'), on= 'NACCID')

In [ ]:
AD_MCI

### Plotting of # visits and Interval

In [ ]:
#plot the number of visits and interval beetween for the two catergories; 
f, axes = plt.subplots(1, 2, figsize=(12, 5))

sns.boxenplot(x="NACCAVST", y="visit_interval", hue="target",showfliers=0,
            data=AD_MCI,ax=axes[0]); axes[0].set(ylim=(200, 1500))

sns.countplot(x="NACCAVST", hue="target",
            data=AD_MCI.groupby(['NACCID']).mean(),ax=axes[1]);
#g.set_xticklabels(g.get_xticklabels(),rotation=45)
labels = [label.get_text() for label in axes[1].get_xticklabels()];
axes[1].set_xticklabels(map(lambda x: "{:g}".format(float(x)), labels));



## Consolidate variables, there is data only recorded in 1st visit for some forms

In [ ]:
#making copy and mask of original nans
AD_MCI_copy=AD_MCI.copy()
mask=(AD_MCI.isnull())


In [ ]:
#replace -4 by nan to be able to back fill
AD_MCI_copy.replace(to_replace=-4, value=np.nan, inplace=True)

AD_MCI_copy=AD_MCI_copy.groupby(['NACCID']).transform(lambda x: x.fillna(method='ffill'))

grouped=AD_MCI_copy.groupby(['NACCID']).fillna(method='ffill') #fastest
grouped

In [ ]:
#add ID back, and replace the nans that were not nan from the beginning to -4
grouped['NACCID']=AD_MCI['NACCID']

grouped.replace(to_replace=np.nan, value=-4, inplace=True)

grouped[mask]=np.nan

In [ ]:
#checking everything is in order
print(np.sum((grouped==-4).unstack()))
np.sum((AD_MCI==-4).unstack())

In [ ]:
grouped.shape

In [ ]:
# drop everything that is not MCI & dem (we won't take into account healthy history. 
#Reconsider this step if building historic model)
clean_AD_MCI=grouped.loc[(grouped['NACCUDSD']>=3)]
print(clean_AD_MCI.shape)
#drop NACCIDs with just one visit
clean_AD_MCI=clean_AD_MCI[clean_AD_MCI['NACCID'].map(clean_AD_MCI['NACCID'].value_counts()) >= 2]
print(clean_AD_MCI.shape)
# drop everything that is not MCI 
clean_AD_MCI=clean_AD_MCI.loc[(clean_AD_MCI['NACCUDSD']==3)]
print(clean_AD_MCI.shape)

#check again to see if we need to drop desangelados
#todrop=[]
c=0
for i in nod_index:
    if clean_AD_MCI.loc[clean_AD_MCI.NACCID == i, 'NACCID'].count()<2 & clean_AD_MCI.loc[clean_AD_MCI.NACCID == i, 'NACCID'].count()> 0:
        c+=1
        #todrop.append(clean_AD_MCI.index[clean_AD_MCI['NACCID'] == i].tolist())
        #clean_AD_MCI=clean_AD_MCI.drop(clean_AD_MCI.index[clean_AD_MCI['NACCID'] == i])

print(c)


In [ ]:
#recheck distros
f, axes = plt.subplots(1, 2, figsize=(12, 5))

sns.boxenplot(x="NACCAVST", y="visit_interval", hue="target",showfliers=0,
            data=clean_AD_MCI,ax=axes[0]); axes[0].set(ylim=(200, 1500))

sns.countplot(x="NACCAVST", hue="target",
            data=clean_AD_MCI.groupby(['NACCID']).mean(),ax=axes[1]);
#g.set_xticklabels(g.get_xticklabels(),rotation=45)
labels = [label.get_text() for label in axes[1].get_xticklabels()];
axes[1].set_xticklabels(map(lambda x: "{:g}".format(float(x)), labels));

In [ ]:
#drop columns full of nans
clean_AD_MCI=clean_AD_MCI.dropna(axis=1,how='all')

In [ ]:
#save this dataframe
#clean_AD_MCI.to_csv('..\data\interim\AD_MCI.csv')